## Synthetic Data (1000 datapoints, SOM map 12x4)

In [ ]:
#generating 3D synthetic data data
def f(X):
    return np.sin(X) + np.random.normal(scale=0.3, size=len(X))

resolution = 1000 # how many datapoints to generate
cycles = 4 # how many sine cycles
length = np.pi * 2 * cycles

X1 = np.arange(0, length, length / resolution)
X2 = np.arange(0, length, length / resolution)
X3 = np.arange(0, length, length / resolution)

synth = np.concatenate(([f(X1)],[f(X2)],[f(X3)]), axis=0).T;

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

synth       = np.load('datasets\synthetic\sinusoidal.npy')
codebook    = np.load('datasets\synthetic\map.npy')

#2-D synusoidal signal
hv.Points(synth[:,0])*hv.Points(synth[:,1])*hv.Points(synth[:,2]).opts(width=900,height=300)

In [ ]:
import somoclu
from pysomvis import PySOMVis
hv.extension("bokeh")

n_rows, n_columns = 12, 4
som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False)
som.train(synth, epochs = 1000)

a = PySOMVis(weights=som.codebook, input_data=synth)
a._mainview

## Climat Data (12 418 days (~35 years), 5 features)

In [2]:
import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler

data = pd.read_excel('datasets\climate\climate_draw_data.xlsx', index_col=0)
data = data.reset_index()

def calc(row):
    return datetime.datetime(int(row[0]), int(row[1]), int(row[2]))

data['time'] = data.apply(calc, axis=1)
data.drop(data.columns[[0, 1, 2, 5, 7, 9]], axis=1, inplace=True)

data['RH2M'] = data.groupby(['RH2M']).apply(lambda x: x[['RH2M']].interpolate())
data['PS'] = data.groupby(['PS']).apply(lambda x: x[['PS']].interpolate())
data['T2M'] = data.groupby(['T2M']).apply(lambda x: x[['T2M']].interpolate())
data['WS50M'] = data.groupby(['WS50M']).apply(lambda x: x[['WS50M']].interpolate())
data['ALLSKY_SFC_LW_DWN'] = data.groupby(['ALLSKY_SFC_LW_DWN']).apply(lambda x: x[['ALLSKY_SFC_LW_DWN']].interpolate())

data = data.set_index(keys='time')
robustscaler = RobustScaler()
for i in range(data.shape[1]):
    x_scaled = robustscaler.fit_transform(data.iloc[:,i].values.reshape(-1,1))
    data.iloc[:,i] = x_scaled
year = np.array(data.loc['1985':'2020'])

In [ ]:
import somoclu
from pysomvis import PySOMVis

n_rows, n_columns = 19, 3
som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False, initialization='pca')
som.train(year, epochs = 10000)

#35 years visualisation
Climat_Data = PySOMVis(weights=som.codebook, input_data=year)
Climat_Data._mainview

## Climat Data (trained SOM map19x3)

In [ ]:
import numpy as np
from pysomvis import PySOMVis

evaldata = np.load('datasets\climate\climate (35 years).npy')
codebook = np.load('datasets\climate\map.npy')

#visualization
Climat_Data2 = PySOMVis(weights=codebook, input_data=evaldata)
Climat_Data2._mainview

## 4 Data Sets, trained by SOMToolbox (http://www.ifs.tuwien.ac.at/dm/somtoolbox/index.html)

In [ ]:
from pysomvis import PySOMVis
from SOMToolBox_Parse import SOMToolBox_Parse

dataset = '10clusters' #iris, chainlink, 10clusters, BostonHousing
inputdata = SOMToolBox_Parse('datasets\\'+dataset+'\\'+dataset+'.vec').read_weight_file()
weights = SOMToolBox_Parse('datasets\\'+dataset+'\\'+dataset+'.wgt.gz').read_weight_file()
classinfo = SOMToolBox_Parse('datasets\\'+dataset+'\\'+dataset+'.cls').read_weight_file()

#visualization
pysomviz = PySOMVis(weights=weights['arr'], m=weights['ydim'], n=weights['xdim'],
                     dimension=weights['vec_dim'], input_data=inputdata['arr'], 
                    classes=classinfo['arr'][:,1], component_names=classinfo['classes_names'])
pysomviz._mainview